<a href="https://colab.research.google.com/github/mdzafri/openvino-workshop/blob/main/LeNet/7_1_MNIST_with_LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST with LeNet
Understanding and Implementing LeNet-5 CNN Architecture

Reference:
1.   [Richmond Alake](https://towardsdatascience.com/understanding-and-implementing-lenet-5-cnn-architecture-deep-learning-a2d531ebc342)
2.   [Jeff Heaton](https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_06_2_cnn.ipynb)

# 1. Dataset Preparation



In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Load the dataset
num_classes = 10
input_shape = (28, 28, 1)

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
# OPTIONAL: Display some dataset samples as an image 
%matplotlib inline
import matplotlib.pyplot as plt
import random

ROWS = 6
random_indices = random.sample(range(x_train.shape[0]), ROWS*ROWS)
sample_images = x_train[random_indices, :]
plt.clf()

fig, axes = plt.subplots(ROWS,ROWS, 
                         figsize=(ROWS,ROWS),
                         sharex=True, sharey=True) 

for i in range(ROWS*ROWS):
    subplot_row = i//ROWS 
    subplot_col = i%ROWS
    ax = axes[subplot_row, subplot_col]

    plottable_image = np.reshape(sample_images[i,:], (28,28))
    ax.imshow(plottable_image, cmap='gray_r')
    
    ax.set_xbound([0,28])

plt.tight_layout()
plt.show()

In [ ]:
# Normalize images to the [0, 1] range
# This is to make the calculations more efficient
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class label vectors to binary class matrices (convert to 1-hot format)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


# 2. Select/Design Model

Choose one of these models to train. DO NOT RUN ALL CELLS HERE. Just choose one, then see the output.

In [ ]:
# 2-layer NN
model = keras.Sequential(
    [
      layers.Flatten(input_shape=(28, 28)),   # Input layer
      layers.Dense(100, activation='relu'),    # Hidden layer(s)
      layers.Dense(num_classes, activation='softmax')  # Output layer
    ]
)
model.summary()

In [ ]:
# CNN LeNet model
model = keras.Sequential(
    [
      keras.Input(shape=input_shape)
      layers.Conv2D(32, kernel_size=(3, 3), activation='relu'), #C1
      layers.MaxPooling2D(pool_size=(2, 2)), #S2
      layers.Conv2D(32, kernel_size=(3, 3), activation='relu'), #C3
      layers.MaxPooling2D(pool_size=(2, 2)), #S4
      layers.Flatten(), #Flatten
      layers.Dense(64, activation='relu'), #C5
      layers.Dense(num_classes, activation='softmax') #Output layer
    ]
)
model.summary()

SyntaxError: ignored

In [ ]:
# LeNet-5 model
model = keras.Sequential(
    [
      layers.Conv2D(10, kernel_size=5, strides=1,  activation='relu', padding='same', input_shape=x_train[0].shape), #C1
      layers.AveragePooling2D(), #S2
      layers.Conv2D(16, kernel_size=5, strides=1, activation='relu', padding='valid'), #C3
      layers.AveragePooling2D(), #S4
      layers.Flatten(), #Flatten
      layers.Dense(120, activation='relu'), #C5
      layers.Dense(84, activation='relu'), #F6
      layers.Dense(num_classes, activation='softmax') #Output layer
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 10)        260       
_________________________________________________________________
average_pooling2d (AveragePo (None, 14, 14, 10)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 16)        4016      
_________________________________________________________________
average_pooling2d_1 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               48120     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

# 3. Train the model

In [ ]:
# set the loss, optimizer and metrics
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.01), metrics=["accuracy"])

# train/fit the model
model.fit(x_train, y_train, batch_size=128, epochs=5, validation_split=0.1)

Epoch 1/5
422/422 [==============================] - 3s 5ms/step - loss: 0.0479 - accuracy: 0.9864 - val_loss: 0.0534 - val_accuracy: 0.9867
Epoch 2/5
422/422 [==============================] - 2s 5ms/step - loss: 0.0394 - accuracy: 0.9888 - val_loss: 0.0530 - val_accuracy: 0.9888
Epoch 3/5
422/422 [==============================] - 2s 5ms/step - loss: 0.0359 - accuracy: 0.9896 - val_loss: 0.0595 - val_accuracy: 0.9883
Epoch 4/5
422/422 [==============================] - 2s 5ms/step - loss: 0.0311 - accuracy: 0.9917 - val_loss: 0.0618 - val_accuracy: 0.9872
Epoch 5/5
422/422 [==============================] - 2s 5ms/step - loss: 0.0375 - accuracy: 0.9896 - val_loss: 0.0755 - val_accuracy: 0.9858


In [ ]:
# Evaluate the trained model performance
score = model.evaluate(x_test, y_test)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

313/313 [==============================] - 1s 3ms/step - loss: 0.0699 - accuracy: 0.9860
Test loss: 0.06985359638929367
Test accuracy: 0.9860000014305115


# 4. Test the trained model

### Make a canvas for user to draw a digit

In [ ]:
# Make a canvas for user to draw a digit
# then save the drawing as a png file
# source: https://gist.github.com/korakot/8409b3feec20f159d8a50b0a811d3bca
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode

canvas_html = """
<canvas width=%d height=%d></canvas>
<button>Finish</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')
ctx.fillStyle = "white";
ctx.fillRect(0, 0, canvas.width, canvas.height);
ctx.lineWidth = %d
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}

canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}

var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})
</script>
"""

def draw(filename='drawing.png', w=150, h=150, line_width=10):
  display(HTML(canvas_html % (w, h, line_width)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
    print("image saved as: ")
    print(filename)
  # return len(binary)

draw()

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image, ImageOps # import pillow image manipulation tool

# Load the image to be tested
user_image = Image.open('drawing.png')
user_image = ImageOps.grayscale(user_image)
user_image = ImageOps.invert(user_image)

# Resize to input_shape
user_image = user_image.resize((input_shape[0],input_shape[1]))
plt.imshow(user_image)
user_image = np.array(user_image).astype("float32") / 255
# user_image = np.expand_dims(user_image, axis=0)
user_image = user_image.reshape(-1, 28, 28, 1)
# print("user_image shape:", user_image.shape)

# Predict the class of the drawing 
result = model.predict(user_image)
print(result)
result = np.argmax(result,axis=1)
print("The AI thinks this is the number:", result[0])
